# **Main Code**
*Updated 08/12/2021*

### ✍ **Patch Notes v1.0**

**Format**
- Python code (.py)
- Code contains a function
- Predefined function name for model answer
- Need to manually put in testing inputs that coresponds to func input
- Function to read any specific line

**Utility**
- currently only works for functions
- Get Model Answer
- Compare Model Answer with students
- Tell whether the program runs
- Count number of corrects
- Display in readable format

---

### ✍ **Patch Notes v2.0**

**Format**
- upload the codes in the correct places
- codes for teacher, test inputs and students can be any name
- Requires test input file 
    1. For **function**, tests list is a 2D list (number of different tests & test inputs)
    2. For **non-function**, requires input of variables to check (eg. ["A", "B"])

**Utility**
- Code does not crash for unsupported format

For **function** question
- Automatically find function name within `teacher_code`
- Generate 'Model Answer' from `teacher_code`
- Run the function in student code using test_inputs conditions
    1. Doesnt Run: Traceback implementation to save ErrorType error line.
    2. Run: Store output vale in dictonary called `codes_output_dict`

For **non-function** question
- Using the `check_var` specified in `test_inputs`
- Store those variable from teacher code for model answer
- Store those variable from student code in `student_output_dict`.


**Other Notes**
- Created `teacher_code` and `test_input` files for non-function format.

---

### ✍ **Patch Notes v2.1**

**New Updates**

- Since the previous year codes has not been set yet, more sample codes has been recreated to show the `student_list` function more clearly.

- Categorise results into 3 categories: 

      "0" = code does not run
      "1" = code runs but is incorrect
      "2" = code runs and is correct

- Created custom functions:
      
      print_result(student_filename) 
    This function prints the detailed results for the specific student.
      
      student_list(category_number, student_codes)
    This function lists out all the student that lies in that result category
    

---
### ✍ **Patch Notes v2.2**

**New Updates**

- Created a function that detects while loop from any submitted code.
- After while loop is detected, the code automatically add lines of codes to count the number of times the loop ran.
- Add print at the end of code to display the loop count.
- This appies for all the while loop in the code.
---

# 📃 **Roadmap** 
*Since 30/11/2021*

✅ Error Detection :
  - detect what kind of error when unable to run
  - which line error occured on 

✅ Situation where code is not in a function format
  - How to detect what to check when there is no clear 'return'

✅ Comparing model answer vs students
  - Seperate into three categories (1) Perfect (2) Run but logic error (3) Does not run

⬜ Dealing with Logic Error
  - similarity of output with perfect output? (ML?)
  - how can automated system help marker in any way.

⬜ ***(New)*** Creating interface for interaction with inputs by teacher. 
  - Function that dispaly desired result_category ✅ 
        def filter(results, result_category)

  - Function that dispaly all data collected on 1 student code ✅ 
        def show_detailed_results(student) 

  - Other desirable functions # still underdevelopment
  - Storing the results and using else where?
  
⬜ ***(New)*** Better Result Visualisation (Final Stages of Program)
  - Maybe a popup application with interactive user interface
  - Choose what function to call and put down inpupts
  - Better output format.




################################################################################

## **Function Question Example**
*Reference: ME1 Computing Progress Test (Iteration 1) Q3*

`Write a function, Flipping, in the space below, that receives a list of items, and returns the same list in reverse order (flipped positions). (Max 6 lines of codes)`

## **Not Function Question Example**
*Reference: ME1 Computing Exercise Sheet 2 Task A1*

`Create manually two arrays A and B, with integer values from 10 to 20 and from 
20 to 30, respectively`






In [ ]:
# Hard Reset
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
from IPython.display import Markdown, display
from google.colab import files
import os.path
import sys, traceback

def printmd(string):
    display(Markdown(string))

def clear_files():
  !rm *.py
  return

"""
Import files in batches:
    1. upload teacher code (can be named anything)
    2. upload test_inputs (consist of testing inputs for model function and teacher code (prenamed: "tests")
    3. upload any amount of student codes (function name should correspond with teacher_code)

Returns:
    file name of teacher_code (string), test_inputs (string), student_codes (list of string)
"""
def import_files():
  printmd('Upload **teacher code file** here')
  uploaded1 = files.upload()
  # take only first code
  uploaded1_keys = []
  for key in uploaded1.keys():
      uploaded1_keys += [str(key)]
  teacher_code = uploaded1_keys[0]
  print('...')

  printmd('Upload **test input file** here')
  uploaded2 = files.upload()
  uploaded2_keys = []
  for key in uploaded2.keys():
      uploaded2_keys += [str(key)]
  test_inputs = uploaded2_keys[0]
  print('...')

  printmd('Upload code files of **all students** here')
  uploaded3 = files.upload()
  student_codes = []
  for key in uploaded3.keys():
      student_codes += [str(key)]

  return teacher_code, test_inputs, student_codes

def find_function_name(teacher_code):
    found_function = 0
    function_name = "function not found"
    f = open(teacher_code)
    lines=f.readlines()
    for i in range(0,len(lines)):
        if lines[i].find("def") != -1:
            start = 4 # function name starts at 5th character [def + space + function name]
            for j in range(4,len(lines[i])):
                if lines[i][j] == "(" :
                    end = j
                    function_name = lines[i][start:end]
                    found_function = 1
                    break
            break
    return found_function ,function_name

def run_function(filename):
    import test_inputs
    exec(open(filename).read())
    result = []
    for test in test_inputs.tests:
            # tests are defined in test_input code.
            # function name was found using func_name
            result += [locals()[function_name](test)]
    return result

def run_file(filename):
    # the following imports check_var
    exec(open(test_inputs).read())
    exec(open(filename).read())
    result = []
    for var in check_var:
        result += [locals()[var]]
    return result

def generate_model_answer(teacher_code, test_inputs, found_function):
    print("Generating model answers...")
    print("...")
    if found_function == 1:
        model_answer = run_function(teacher_code)
        print("Results =" + str(model_answer))
    else:
        model_answer = run_file(teacher_code)
    return model_answer

# creates students_dict consisting of student file names and results
def generate_student_outputs(student_codes, test_inputs, found_function):
    output_dict = {}
    for code in student_codes:
        print("\n")
        print("Testing " + str(code))
        print("...")
        if found_function == 1:
            try:
                output_dict[code] = run_function(code)
                print("Results =" + str(output_dict[code]))
            except:
                error_tb = traceback.format_exc()
                output_dict[code] = "Error \n" + str(error_tb)
                print("ERROR: Cannot run " + str(code))
                print(error_tb)
        else:
            try:
                output_dict[code] = run_file(code)
                print("Results =" + str(output_dict[code]))
            except:
                error_tb = traceback.format_exc()
                output_dict[code] = "Error \n" + str(error_tb)
                print("ERROR: Cannot run " + str(code))
                print(error_tb)
    return output_dict

def categorise_results(model_answer,student_codes, output_dict):
    result_dict = {}
    for student in output_dict:
        if output_dict[student] == model_answer:
            result_dict[student] = 2  # Run and correct
        elif "Error" in output_dict[student]:
            result_dict[student] = 0  # Does not Run
        else:
            result_dict[student] = 1  # Run but incorrect
    return result_dict

In [ ]:
## MAIN CODE ##

# clear previous python files
clear_files()

# import files and save into 3 batches of filenames
teacher_code, test_inputs, student_codes = import_files()

# Check if teacher code (exam question) is a function or not
found_function, function_name = find_function_name(teacher_code)

print("Finding function in " + str(teacher_code))
print("...")
print("found function = " + str(found_function))
printmd("function name = **" + str(function_name) + "**")
print("...")

# create a dictionary to store answers (both model and student answers)  
model_answer = generate_model_answer(teacher_code, test_inputs, found_function)
output_dict = generate_student_outputs(student_codes, test_inputs, found_function)

# create dict to seperate student results into different categories
    # result_category = 0  -> code does not run
    # result_category = 1  -> code runs but is incorrect
    # result_category = 2  -> code runs and is correct

result_dict = categorise_results(model_answer,student_codes, output_dict)

# dealing with logical errors..



# Visualising Results
printmd("**Final Results**")
for student in student_codes:
    print("---")
    print(student)
    print("\033[1m currect answer = \033[0m" + str(model_answer))
    print("\033[1m student output = \033[0m" + str(output_dict[student]))
    print("\033[1m result category = \033[0m" + str(result_dict[student]))




rm: cannot remove '*.py': No such file or directory


Upload **teacher code file** here

Saving teacher_code.py to teacher_code.py
...


Upload **test input file** here

Saving test_inputs.py to test_inputs.py
...


...

Saving exceptionCode1.py to exceptionCode1.py
Saving exceptionCode2.py to exceptionCode2.py
Saving fullMarkCode1.py to fullMarkCode1.py
Saving fullMarkCode2.py to fullMarkCode2.py
Saving fullMarkCode3.py to fullMarkCode3.py
Saving logicErrorCode1.py to logicErrorCode1.py
Finding function in teacher_code.py
...
found function = 1


function name = **flip**

...
Generating model answers...
...
Results =[[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, -1]]


Testing exceptionCode1.py
...
ERROR: Cannot run exceptionCode1.py
Traceback (most recent call last):
  File "<ipython-input-1-fe0939404086>", line 103, in generate_student_outputs
    output_dict[code] = run_function(code)
  File "<ipython-input-1-fe0939404086>", line 72, in run_function
    result += [locals()[function_name](test)]
  File "<string>", line 7, in flip
NameError: name 'flippedd' is not defined



Testing exceptionCode2.py
...
ERROR: Cannot run exceptionCode2.py
Traceback (most recent call last):
  File "<ipython-input-1-fe0939404086>", line 103, in generate_student_outputs
    output_dict[code] = run_function(code)
  File "<ipython-input-1-fe0939404086>", line 72, in run_function
    result += [locals()[function_name](test)]
  File "<string>", line 7, in flip
NameError: name 'flippedd' is not defined



Testing fullMarkCode1.py
...
Results =[[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, 

**Final Results**

---
exceptionCode1.py
 currect answer = [[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, -1]]
 student output = Error 
Traceback (most recent call last):
  File "<ipython-input-1-fe0939404086>", line 103, in generate_student_outputs
    output_dict[code] = run_function(code)
  File "<ipython-input-1-fe0939404086>", line 72, in run_function
    result += [locals()[function_name](test)]
  File "<string>", line 7, in flip
NameError: name 'flippedd' is not defined

 result category = 0
---
exceptionCode2.py
 currect answer = [[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, -1]]
 student output = Error 
Traceback (most recent call last):
  File "<ipython-input-1-fe0939404086>", line 103, in generate_student_outputs
    output_dict[code] = run_function(code)
  File "<ipython-input-1-fe0939404086>", line 72, in run_function
    result += [locals()[function_name](test)]
  File "<string>", line 7, in flip
NameError: name 'flippedd' is not defined

 result category = 0
---
fullMarkCode1.py
 currect answer = [[4, 

# **Custom Outputs**

In [ ]:
# print out results for specefic student

def print_result(student_filename):
    print("------------------------------------------------------------------")
    print("\033[1m filename = \033[0m" + student_filename)
    print("\033[1m currect answer = \033[0m" + str(model_answer))
    print("\033[1m student output = \033[0m" + str(output_dict[student_filename]))
    print("\033[1m result category = \033[0m" + str(result_dict[student_filename]))
    print("------------------------------------------------------------------")
    return

# student_full_mark_code.py
print("please input student filename here")
valid = 0
while valid == 0:
    student_filename = input()
    try:
        output_dict[student_filename]
        valid = 1
    except:
        print("\033[1mError:\033[0m Please input a valid file name \n")

print_result(student_filename)

please input student filename here
fullMarkCode2.py
------------------------------------------------------------------
 filename = fullMarkCode2.py
 currect answer = [[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, -1]]
 student output = [[4, 3, 2, 1], [0, 0, 2, 1], [2, 1, -5, -1]]
 result category = 2
------------------------------------------------------------------


In [ ]:
# print out students who are in the category: 0, 1, 2

def student_list(category_number, student_codes):

    count = 1
    for student in student_codes:
        if result_dict[student] == category_number:
            print(str(count) + ". " + student)
    return

print("please input result category number: ")
print("0 = code does not run ")
print("1 = code runs but has logical error ")
print("2 = code runs perfectly ")

valid = 0
while valid == 0:
    category_number = int(input())
    if category_number == 0:
        print("\n")
        print("\033[1mThe follow codes do not run: \033[0m")
        valid = 1 
    elif category_number == 1:
        print("\n")
        print("\033[1mThe follow codes run but contains logical error: \033[0m")
        valid = 1 
    elif category_number == 2:
        print("\n")
        print("\033[1mThe follow codes runs perfectly: \033[0m")
        valid = 1 
    else:
        print("\033[1mError\033[0m: Please input a valid category number")

student_list(category_number, student_codes)

please input result category number: 
0 = code does not run 
1 = code runs but has logical error 
2 = code runs perfectly 
1


The follow codes run but contains logical error: 
1. logicErrorCode1.py


In [ ]:
# Code that reads student code, add counter at every while loop and runs the edited code.
# import sample whileloop code
from google.colab import files
!rm *.py

print("Please upload while-loop test file here")
uploaded = files.upload()
uploaded_keys = []
for key in uploaded.keys():
    uploaded_keys += [str(key)]
filename = uploaded_keys[0]

# for bold display #
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def visualise_code(filename):
    printmd("**Visualising the input code**")
    f=open(filename)
    lines=f.readlines()
    for i in range(0,len(lines)):
      print("#" + str(i) + " " + lines[i])
    printmd("**End Visualisation**")
    print("\n")

def find_whileloop(filename):
    edited_code = ""

    whileloop_count = 0  
    counter = 0

    f = open(filename)
    lines=f.readlines()
    for i in range(len(lines)):
        edited_code += lines[i]
        if i > (len(lines) -2):
            break
        if lines[i].find("while") != -1:  #this will add a line after while loop
            edited_code += ("    counter" + str(counter) +  " += 1\n")
        if lines[i+1].find("while") != -1:  #this will add a line before while loop
            whileloop_count += 1
            counter += 1
            edited_code += ("counter" + str(counter) +  " = 0\n")
    
    # after added all lines
    if whileloop_count > 0:
        edited_code += ("\n")
        for j in range(1,counter+1):
            edited_code += ("print(counter" +str(j)+")\n")
        return edited_code, whileloop_count

def runcode(codelines):
    # need to join all the lines into 1 string for exec() to work
    long_string = ("")
    for line in codelines:
        long_string += line
    try:
        exec(long_string)
    except:
        printmd("**There is an error, codelines cannot be executed**")

### MAIN ###

# filename = filename (defined previously)
visualise_code(filename)
edited_code, whileloop_count = find_whileloop(filename)

print(find_whileloop(filename))

printmd("**Start of Edited Code**")
print(edited_code)
printmd("**End of Edited Code**")
print("\n")
printmd("**Running Edited Code**")
runcode(edited_code)
printmd("**Completed!**")

Please upload while-loop test file here


Saving while_loop3.py to while_loop3.py


**Visualising the input code**

#0 #  write the sum of digits for any positive input

#1 

#2 x = 0

#3 

#4 while x<10: # should run 10 times

#5     x+=1

#6 

#7 y = 5

#8 

#9 while y>1: # should run 4 times

#10     y-=1

#11 

#12 

#13 z = 0

#14 

#15 while z == 0: # should run 1 time

#16     z +=1



**End Visualisation**

...

#  write the sum of digits for any positive input

x = 0

counter1 = 0
while x<10: # should run 10 times
    counter1 += 1
    x+=1

y = 5

counter2 = 0
while y>1: # should run 4 times
    counter2 += 1
    y-=1


z = 0

counter3 = 0
while z == 0: # should run 1 time
    counter3 += 1
    z +=1

print(counter1)
print(counter2)
print(counter3)



**End of Edited Code**

**Running Edited Code**

10
4
1


**Completed!**